In [1]:
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms
from torchvision.utils import make_grid
from torch.utils.data.dataloader import DataLoader
import matplotlib.pyplot as plt

In [2]:
# Apply transform to Data
train_transform = transforms.Compose([
                transforms.RandomResizedCrop(size=256, scale=(0.8, 1.0)),
                transforms.RandomRotation(degrees=15),
                transforms.RandomHorizontalFlip(),
                transforms.CenterCrop(size=224),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406],
                                     [0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
                transforms.Resize(size=256),
                transforms.CenterCrop(size=224),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406],
                                     [0.229, 0.224, 0.225])
])

In [3]:
train_data = torchvision.datasets.CIFAR100(root='data',
                                          train=True,
                                          transform=train_transform,
                                          download=True)
test_data = torchvision.datasets.CIFAR100(root='data',
                                         train=False,
                                         transform=test_transform)

Files already downloaded and verified


In [4]:
batch_size = 32
train_loader = DataLoader(train_data,
                        batch_size=batch_size,
                        shuffle=True)
test_loader = DataLoader(test_data,
                        batch_size=batch_size,
                        shuffle=True)

In [5]:
print(f'Kich thuoc cua tap train la: {len(train_loader)}')
print(f'Kich thuoc cua tap test la: {len(test_loader)}')

Kich thuoc cua tap train la: 1563
Kich thuoc cua tap test la: 313


In [6]:
import os
import numpy as np
import time

In [7]:
#sample_idx = torch.randint(len(train_data), size=(1,)).item()
#img, label = train_data[sample_idx]
#plt.imshow(img.permute(1,2,0))
#print(label)

In [8]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [9]:
# Load pretrained ResNet50 Model
resnet50 = torchvision.models.resnet50(pretrained=True)
resnet50 = resnet50.to(device)

In [10]:
# Freeze layers 1-6 in total 10 layers of ResNet
ct = 0
for child in resnet50.children():
    ct += 1
    if ct < 7:
        for param in child.parameters():
            param.requires_grad = False

In [11]:
from torchsummary import summary
# Change the final layer of ResNet50 Model for Transfer Learning
fc_inputs = resnet50.fc.in_features

resnet50.fc = nn.Sequential(nn.Linear(fc_inputs, 256),
                           nn.ReLU(),
                           nn.Dropout(0.4),
                           nn.Linear(256, 100), #Since 100 possible outputs
                           nn.LogSoftmax(dim=1))
# Convert model to be used on GPU
resnet50 = resnet50.to(device)

In [12]:
#summary(resnet50, (3,224,224))

In [13]:
#print(resnet50)

In [15]:
train_data_size = len(train_data)
test_data_size = len(test_data)

In [19]:
def train_and_valid(model, loss_criterion, optimizer, scheduler, epochs=30):
    
    history = []
    best_loss = 100000.0
    best_epoch = None
    
    for epoch in range(epochs):
        start = time.time()
        # Set to train model
        model.train()
        
        train_loss = 0.0
        train_acc = 0.0
        
        valid_loss = 0.0
        valid_acc = 0.0
        
        for i, (inputs, labels) in enumerate(train_loader):
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            # Clean exist gradient
            optimizer.zero_grad()
            # Forward pass - compute outputs on input data using the model
            outputs = model(inputs)
            # Compute loss
            loss = loss_criterion(outputs, labels)
            # Backpropagate the gradients
            loss.backward()
            # Update parameter
            optimizer.step()
            scheduler.step()
            
            # Compute the total loss for the batch and add it to train_loss
            train_loss += loss.item()*inputs.size(0)
            # Compute the accuracy
            ret, predictions = torch.max(outputs.data, 1)
            correct_counts = predictions.eq(labels.data.view_as(predictions))
            
            # Convert correct_counts to float and then compute the mean
            acc = torch.mean(correct_counts.type(torch.FloatTensor))
            
            # Compute total accuracy in the whole batch and add to train_acc
            train_acc += acc.item() * inputs.size(0)
            
        with torch.no_grad():
            
            # set to evaluation mode
            model.eval()
            
            # Validation loop
            for j, (inputs, labels) in enumerate(test_loader):
                inputs = inputs.to(device)
                labels = labels.to(device)
                
                # Forward pass
                loss = loss_criterion(outputs, labels)
                
                # Compute the total loss for the batch and add it to valid_loss
                
                valid_loss += loss.item() * inputs.size(0)
                # Calculate validation accuracy
                ret, predictions = torch.max(outputs.data, 1)
                correct_counts = predictions.eq(labels.data.view_as(predictions))

                # Convert correct_counts to float and then compute the mean
                acc = torch.mean(correct_counts.type(torch.FloatTensor))

                # Compute total accuracy in the whole batch and add to valid_acc
                valid_acc += acc.item() * inputs.size(0)
                
        if valid_loss < best_loss:
            best_loss = valid_loss
            best_epoch = epoch
            
        # Find average training loss and training accuracy
        avg_train_loss = train_loss/train_data_size 
        avg_train_acc = train_acc/train_data_size

        # Find average training loss and training accuracy
        avg_valid_loss = valid_loss/test_data_size 
        avg_valid_acc = valid_acc/test_data_size
        
        history.append([avg_train_loss, avg_valid_loss, avg_train_acc, avg_valid_acc])
        
        epoch_end = time.time()
        
        print('Epoch: [{}/{}]\t Train_loss: {}\t Train_acc: {}\t Valid_loss: {}\t Valid_acc: {}\t Time: {}'
              .format(epoch+1, epochs, avg_train_loss, avg_train_acc, avg_valid_loss, avg_valid_acc, epoch_end-start))
        
        # Save if the model has best accuracy till now
        torch.save(model, 'model_'+str(epoch)+'.pt')
        
    return model, history, best_epoch

In [20]:
num_epochs = 30
# Define Optimizer and Loss Function
loss_func = nn.NLLLoss()
optimizer = torch.optim.Adam(resnet50.parameters(), lr=0.005)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[10,20], gamma=0.1)
trained_model, history, best_epoch = train_and_valid(resnet50, loss_func, optimizer, scheduler, num_epochs)
torch.save(history, 'history.pt')

KeyboardInterrupt: 